In [30]:
from argparse import Namespace
import xml.etree.ElementTree as ET
import os
from json import dumps

In [29]:
pdk = 'sky130A'
gf_pdk_option = 'D'

# --- parser starts here ---

os.environ['PDK'] = pdk
if pdk != 'gf180mcu':
    gf_pdk_option = ''
os.environ['GF_PDK_OPTION'] = gf_pdk_option

lyt  = '$PDK_ROOT/$PDK$GF_PDK_OPTION/libs.tech/klayout/tech/$PDK.lyt'
tech = ET.parse(os.path.expandvars(lyt)).getroot()

name  = tech.find('name').text
desc  = tech.find('description').text
dbu   = float(tech.find('dbu').text)

base_dirpath = tech.find('original-base-path').text
lyp_filename  = tech.find('layer-properties_file').text
lyp          = os.path.join(base_dirpath, lyp_filename)

add_other_layers = tech.find('add-other-layers').text == 'true'

default_grids  = tech.find('default-grids')
force_grids    = '!' in default_grids
default_grids  = default_grids.text.replace('!', '').split(',')
maingrid_step, subgrid_step = map(float, default_grids)

gds2 = tech.find('./reader-options/gds2')
box_mode = int(gds2.find('box-mode').text)
allow_big_records = gds2.find('allow-big-records').text == 'true'
allow_multi_xy    = gds2.find('allow-multi-xy-records').text == 'true'

lefdef = tech.find('./reader-options/lefdef')
lef_files = lefdef.find('lef-files').text if lefdef is not None else None
produce_net_names = lefdef.find('produce-net-names').text == 'true'

writer_gds2 = tech.find('./writer-options/gds2')
write_timestamps = writer_gds2.find('write-timestamps').text == 'true'
max_vertex_count = int(writer_gds2.find('max-vertex-count').text)

connectivity = tech.find('./connectivity')
connections = [c.text for c in connectivity.findall('connection')]
symbols     = {s.attrib.get('name', s.tag): s.text for s in connectivity.findall('symbols')}

print(f"""
Technology:
  Name             = "{name}"
  Description      = "{desc}"
  dbu              = {dbu}
  lyp              = "{lyp}"
  add_other_layers = {add_other_layers}
  maingrid_step    = {maingrid_step}
  subgrid_step     = {subgrid_step}
  force_grids      = {force_grids}
  
Reader Options (GDS2):
  box_mode          = {box_mode}
  allow_big_records = {allow_big_records}
  allow_multi_xy    = {allow_multi_xy}

Reader Options (LEF/DEF):
  lef_files          = {lef_files}
  produce_net_names = {produce_net_names}

Writer Options (GDS2):
  write_timestamps = {write_timestamps}
  max_vertex_count = {max_vertex_count}

Connectivity:
  Connections = {dumps(connections, indent=4)}
  Symbols     = {dumps(symbols, indent=4)}
""")


Technology:
  Name             = "sky130"
  Description      = "SkyWater 130nm technology"
  dbu              = 0.001
  lyp              = "$PDK_ROOT/$PDK/libs.tech/klayout/sky130A.lyp"
  add_other_layers = True
  maingrid_step    = 0.01
  subgrid_step     = 0.005
  force_grids      = False

Reader Options (GDS2):
  box_mode          = 1
  allow_big_records = True
  allow_multi_xy    = True

Reader Options (LEF/DEF):
  lef_files          = merged.lef
  produce_net_names = True

Writer Options (GDS2):
  write_timestamps = True
  max_vertex_count = 8000

Connectivity:
  Connections = [
    "poly,licon,li",
    "li,mcon,met1",
    "met1,via1,met2",
    "met2,via2,met3",
    "met3,via3,met4",
    "met4,via4,met5"
]
  Symbols     = {
    "symbols": "met5='72/20+72/5-72/14-72/15'"
}

